# There is no bijective 6bit counter

We want to show that it is not possible to enumerate numbers from 0 to 63 (in any order) with a 6 bit circuit in a **bijective** way.       
Indeed a counter is either a bijection or an almost bijection where the loop does not go back to the beginin.  
We check here for bijective counters.   
We don't want to bruteforce over all circuits (there are 2^44 circuits which makes about 17 000 billions).
## How to reduce the search space ?
We reduce our search space with the following method:   
In an enumeration of all 6 bits strings what is the behavior of the two first bits ?    
They will do all 2 bits sequences: 00, 10, 01, 11   
The question is how many times ?   
By symmetry it will be 64/4 = 16 each     
Let's be sure with the following calculation:

In [1]:
import itertools
all6b = list(itertools.product([0,1],repeat=6))
count_first2b = {}
for seq in all6b:
    seq_str2b = str(seq[0])+str(seq[1])
    if not seq_str2b in count_first2b:
        count_first2b[seq_str2b] = 0
    count_first2b[seq_str2b] += 1
for seq_2b in count_first2b:
    print(seq_2b + " " + str(count_first2b[seq_2b]))

00 16
01 16
10 16
11 16


**Hence** the part of our network controlling the first two bits will have to have the property to make appear 00, 01, 10, 11, 16 times each, no matters of the order.    
We call this the **"4*16" property**.    
This part of the network is a function of x_0, x_1 and x_2 that outputs 2 bits.    
Its structure is quite small since we have 4 choices for gate 1, 256 for gate 2 and 16 for gate 3 (as we only take its UP output).   
**Over all we can iterate over all 4x256x16=16384 functions and juste keep the ones that have our "4*16" property.**

## Finding the "4*16" functions
### Encoding circuits
To encode our networks we specify its gates. Here:  
- Gate 1 is a {0,1} -> {0,1} function
- Gate 2 is a {0,1}^2 -> {0,1}^2 function   
- Gate 3 is a {0,1}^2 -> {0,1} function (we only take the UP output)     

We encode these functions in the following way:   

- g1: {0,1} -> {0,1}  will be a 2-tuple of bits (f(0),f(1))
- g2: {0,1}^2 -> {0,1}^2 will be a 4-tuple of 2bits (f(00),f(01),f(10),f(11))
- g3U: {0,1}^2 -> {0,1} will be a 4-tuple of bits (f(00),f(01),f(10),f(11))  

A circuit here will be a 3-tuple (g1,g2,g3U).       

Here we construct all these circuits:     


In [2]:
# This function gives the list of all networks
# inputing x0 x1 x2 and processing for 2 bits output
def list_all_circuits_2b_x012():
    all_g1 = list(itertools.product(['0','1'],repeat=2))
    all_g2 = list(itertools.product(['00','01','10','11'],repeat=4))
    all_g3U = list(itertools.product(['0','1'],repeat=4))
    all_circuits2b_x012 = list(itertools.product(all_g1,all_g2,all_g3U))
    return all_circuits2b_x012
all_2bc_x012 = list_all_circuits_2b_x012()
print("Example of circuit on x1 x2 x3: "+str(all_2bc_x012[0]))

Example of circuit on x1 x2 x3: (('0', '0'), ('00', '00', '00', '00'), ('0', '0', '0', '0'))


Then we define the computation inside these 2bits output networks:

In [3]:
# Given a 2 bits circuit c and an input x
# the following function computes the circuit's output
def output_of_2b_x012(c,x):  
    output_g1 = int(c[0][x[0]])
    output_g3U = int(c[2][2*x[1]+x[2]])
    output_g2 = c[1][2*output_g1+output_g3U]
    return output_g2

**Now** we construct the predicate for a network to have the "4*16" property:

In [4]:
def has_4x16(c, outputing_f):
    seen_2b = {}
    for seq_6b in all6b:
        out = outputing_f(c,seq_6b)
        if not out in seen_2b:
            seen_2b[out] = 0
        seen_2b[out] += 1
    if len(seen_2b) == 4:
        for seq_2b in seen_2b:
            if seen_2b[seq_2b] != 16:
                return False
        return True
    return False

**Now** Let's grab them all:

In [5]:
all_4x16_x012 = []
for c in all_2bc_x012:
    if has_4x16(c,output_of_2b_x012):
        all_4x16_x012.append(c)

In [6]:
print("There are "+str(len(all_4x16_x012))+" 4x16 2b circuits on x0, x1, x2 with 4x16 prop")

There are 288 4x16 2b circuits on x0, x1, x2 with 4x16 prop


### Getting rid of redundancy
But some of them are equivalents in the sense they compute the same function, an other way to **reduce our search space** is to keep one represent of each class of function that is calculated, that what does the following.

In [7]:
unique_4x16_x012 = {}
for c in all_4x16_x012:
    all3b = itertools.product([0,1],repeat=3)
    r = ""
    for seq_3b in all3b:
        r += output_of_2b_x012(c,seq_3b)
    unique_4x16_x012[r] = c
good_4x16_x012 = []
for r in unique_4x16_x012:
    good_4x16_x012.append(unique_4x16_x012[r])
print("It remains "+str(len(good_4x16_x012))+" circuits.")

It remains 72 circuits.


Something **remarkable** is that all the remaining circuits have the same gate 1 which is set to **copy** mode (0,1).  
You can verify it by looking at them all: 

In [8]:
for (i,c) in enumerate(good_4x16_x012):
    print("c"+str(i)+" "+str(c))

c0 (('1', '0'), ('11', '10', '01', '00'), ('1', '1', '0', '0'))
c1 (('1', '0'), ('11', '10', '01', '00'), ('1', '0', '1', '0'))
c2 (('1', '0'), ('11', '10', '01', '00'), ('1', '0', '0', '1'))
c3 (('1', '0'), ('11', '10', '01', '00'), ('0', '1', '1', '0'))
c4 (('1', '0'), ('11', '10', '01', '00'), ('0', '1', '0', '1'))
c5 (('1', '0'), ('11', '10', '01', '00'), ('0', '0', '1', '1'))
c6 (('1', '0'), ('11', '10', '00', '01'), ('0', '0', '1', '1'))
c7 (('1', '0'), ('11', '10', '00', '01'), ('0', '1', '0', '1'))
c8 (('1', '0'), ('11', '10', '00', '01'), ('0', '1', '1', '0'))
c9 (('1', '0'), ('11', '10', '00', '01'), ('1', '0', '0', '1'))
c10 (('1', '0'), ('11', '10', '00', '01'), ('1', '0', '1', '0'))
c11 (('1', '0'), ('11', '10', '00', '01'), ('1', '1', '0', '0'))
c12 (('1', '0'), ('11', '01', '10', '00'), ('1', '1', '0', '0'))
c13 (('1', '0'), ('11', '01', '10', '00'), ('1', '0', '1', '0'))
c14 (('1', '0'), ('11', '01', '10', '00'), ('1', '0', '0', '1'))
c15 (('1', '0'), ('11', '01', '10',

## We do the same for x3,x4,x5 (2 lasts bits)!!

In [9]:
# This function gives the list of all networks
# inputing x3 x4 x5 and processing for 2 bits output
def list_all_circuits_2b_x345():
    all_g5D = list(itertools.product(['0','1'],repeat=4))
    all_g6 = list(itertools.product(['00','01','10','11'],repeat=4))
    all_g7 = list(itertools.product(['0','1'],repeat=2))
    all_circuits2b_x345 = list(itertools.product(all_g5D,all_g6,all_g7))
    return all_circuits2b_x345
all_2bc_x345 = list_all_circuits_2b_x345()
print("Example of circuit on x3 x4 x5: "+str(all_2bc_x345[0]))

Example of circuit on x3 x4 x5: (('0', '0', '0', '0'), ('00', '00', '00', '00'), ('0', '0'))


In [10]:
# Given a 2 bits circuit c and an input x
# the following function computes the circuit's output
def output_of_2b_x345(c,x):  
    output_g5D = int(c[0][2*x[3]+x[4]])
    output_g7 = int(c[2][x[5]])
    output_g6 = c[1][2*output_g5D+output_g7]
    return output_g6

In [11]:
all_4x16_x345 = []
for c in all_2bc_x345:
    if has_4x16(c,output_of_2b_x345):
        all_4x16_x345.append(c)

In [63]:
print("There are "+str(len(all_4x16_x345))+" 4x16 2b circuits on x3, x4, x5")

There are 288 4x16 2b circuits on x3, x4, x5


In [13]:
unique_4x16_x345 = {}
for c in all_4x16_x345:
    all3b = itertools.product([0,1],repeat=3)
    r = ""
    for seq_3b in all3b:
        r += output_of_2b_x345(c,(0,0,0)+seq_3b)
    unique_4x16_x345[r] = c
good_4x16_x345 = []
for r in unique_4x16_x345:
    good_4x16_x345.append(unique_4x16_x345[r])
print("It remains "+str(len(good_4x16_x345))+" circuits on x3, x4, x5 with 4x16 prop.")

It remains 72 circuits on x3, x4, x5 with 4x16 prop.


## Now what ? We do the same for x1,x2,x3,x4 (middle bits) !!!
The 4x16 property should be true also for the two outputs bits of the middle.  
They are ruled by a function of x1,x2,x3,x4 we should again compute all networks!!!

In [14]:
# This function gives the list of all networks
# inputing x3 x4 x5 and processing for 2 bits output
def list_all_circuits_2b_x1234():
    all_g3D = list(itertools.product(['0','1'],repeat=4))
    all_g4 = list(itertools.product(['00','01','10','11'],repeat=4))
    all_g5U = list(itertools.product(['0','1'],repeat=4))
    all_circuits2b_x1234 = list(itertools.product(all_g3D,all_g4,all_g5U))
    return all_circuits2b_x1234
all_2bc_x1234 = list_all_circuits_2b_x1234()
print("There are "+str(len(all_2bc_x1234))+" circuits on x1,x2,x3,x4")

There are 65536 circuits on x1,x2,x3,x4


In [15]:
# Given a 2 bits circuit c and an input x
# the following function computes the circuit's output
def output_of_2b_x1234(c,x):  
    output_g3D = int(c[0][2*x[1]+x[2]])
    output_g5U = int(c[2][2*x[3]+x[4]])
    output_g4 = c[1][2*output_g3D+output_g5U]
    return output_g4

In [16]:
unique_x1234 = {}
for c in all_2bc_x1234:
    all4b = itertools.product([0,1],repeat=4)
    r = ""
    for seq_4b in all4b:
        r += output_of_2b_x1234(c,(0,seq_4b[0],seq_4b[1],seq_4b[2],seq_4b[3],0))
    unique_x1234[r] = c
good_x1234 = []
for r in unique_x1234:
    good_x1234.append(unique_x1234[r])
print("It remains "+str(len(good_x1234))+" circuits on x1, x2, x3, x4.")

It remains 11344 circuits on x1, x2, x3, x4.


In [17]:
good_4x16_x1234 = []
for c in good_x1234:
    if has_4x16(c,output_of_2b_x1234):
        good_4x16_x1234.append(c)

In [18]:
print("There are "+str(len(good_4x16_x1234))+" circuits on x1, x2, x3, x4 with 4x16 prop.")

There are 216 circuits on x1, x2, x3, x4 with 4x16 prop.


# The answer

In [19]:
all_eligible_c = list(itertools.product(good_4x16_x012,good_4x16_x1234,good_4x16_x345))

In [40]:
def output_of_6bc(c,x):
    x = (int(x[0]),int(x[1]),int(x[2]),int(x[3]),int(x[4]),int(x[5]))
    g1 = c[0][0]
    g2 = c[0][1]
    g3U = c[0][2]
    g3D = c[1][0]
    g4 = c[1][1]
    g5U = c[1][2]
    g5D = c[2][0]
    g6 = c[2][1]
    g7 = c[2][2]
    output_g1 = int(g1[x[0]])
    output_g3U = int(g3U[2*x[1]+x[2]])
    output_g3D = int(g3D[2*x[1]+x[2]])
    output_g5U = int(g5U[2*x[3]+x[4]])
    output_g5D = int(g5D[2*x[3]+x[4]])
    output_g7 = int(g7[x[5]])
    #print(g7)
    #print(output_g1)
    #print(output_g3U)
    #print(output_g3D)
    #print(output_g5U)
    #print(output_g5D)
    #print(output_g7)
    #print(g5U)
    #print(g5D)
    
    output_g2 = g2[2*output_g1+output_g3U]
    output_g4 = g4[2*output_g3D+output_g5U]
    output_g6 = g6[2*output_g5D+output_g7]
    #print("ooo"+str(g6))
    
    return output_g2+output_g4+output_g6

In [23]:
how_many_sees = {}
c64 = []
for c in all_eligible_c:
    seen = {}
    for seq_6b in all6b:
        out = output_of_6bc(c,seq_6b)
        seen[out] = 1
    how_many_sees[len(seen)] = c
    if len(seen) == 64:
        c64.append(c)

In [24]:
print(how_many_sees)
print(len(c64))

497664


In [58]:
def to_dec(seq_6b):
    r = 0
    for (i,b) in enumerate(seq_6b):
        r += int(b)*(2**i)
    return r

def how_many_orbits(c,p=False,o_size=[]):
    nb_o = 0
    seen_x = {}
    
    for seq_6b in all6b:
        curr_state = seq_6b
        k = 0
        while not to_dec(curr_state) in seen_x:
            if k==0:
                #print("orbite")
                nb_o +=1
            k += 1     
            seen_x[to_dec(curr_state)] = 1
            curr_state = output_of_6bc(c,curr_state)
        if p and k != 0:
            o_size.append((curr_state,k))
    return nb_o

In [41]:
orbits = {}
for c in c64:
    no = how_many_orbits(c)
    if not no in orbits:
        orbits[no] = []
        print(no)
    orbits[no].append(c)

64
32
48
16
40
24
8
28
12
20
4
36
22
6
14
18
2
10


# Conclusion
All bijection have at least 2 orbits so there's no counter

In [46]:
def circuit_to_c(c):
    print(c[0][0][0]+c[0][0][1])
    print(c[0][1][0]+c[0][1][1]+c[0][1][2]+c[0][1][3])
    r = ""
    for i in range(4):
        r += c[0][2][i]+c[1][0][i]
    print(r)
    print(c[1][1][0]+c[1][1][1]+c[1][1][2]+c[1][1][3])
    r = ""
    for i in range(4):
        r += c[1][2][i]+c[2][0][i]
    print(r)
    print(c[2][1][0]+c[2][1][1]+c[2][1][2]+c[2][1][3])
    print(c[2][2][0]+c[2][2][1])

In [64]:
circuit_to_c(((('1', '0'), ('01', '00', '10', '11'), ('1', '1', '0', '0')),
  (('1', '0', '0', '1'), ('01', '00', '10', '11'), ('1', '1', '0', '0')),
  (('1', '0', '0', '1'), ('01', '00', '10', '11'), ('1', '0'))))

10
01001011
11100001
01001011
11100001
01001011
10
